In [2]:
import pandas as pd
import numpy as np

In [3]:
analytics_assessment = pd.read_excel("data/Analytics Assessment Report 2024-07-01 2024-07-31.xlsx")
#analytics_assessment.head()

In [4]:
def duration_to_seconds(input):
    if input == 0:
        return 0
    else:
        h, m, s = input.split()
        return int(h[:-1]) * 3600 + int(m[:-1])*60 + int(s[:-1])

analytics_assessment["duration in seconds"] = analytics_assessment["Duration (exact)"].apply(duration_to_seconds)
# analytics_assessment.head()


In [5]:
patients_by_duration = analytics_assessment.groupby(by=["Patient Id", "Enrolled Care Program"])["duration in seconds"].sum().reset_index()
merged_df = pd.merge(patients_by_duration, 
                     analytics_assessment[['Patient Id', 'Pharmacy Name', 'POD', 'User Name (First then Last)']].drop_duplicates(), 
                     on="Patient Id", 
                     how="left")
merged_df.head()

,Patient Id,Enrolled Care Program,duration in seconds,Pharmacy Name,POD,User Name (First then Last)
0,2583.0,CCM,2400,iVira,NaN,Francis Fuentes
1,2583.0,CCM,2400,iVira,Milford Pod,Michelle Mazry
2,2583.0,CCM,2400,iVira,Alex Pod,Michelle Mazry
3,2612.0,CCM,3911,iVira,Alex Pod,Jay Patel
4,2612.0,PCM,96,iVira,Alex Pod,Jay Patel


In [6]:
# this will calculate revenue per minute left 
# args: duration (in seconds)
# output: ["revenue per minute"], ["remaining minutes"] - How much revenue you can gain per minute of work and how many minutes you need to work to achieve next price target
def revenue_per_minute(program, duration):
    if program == "RPM":
        if duration >= 4800: # this is the maxed out case
            return [0, 0, 0, 0]
        billing_bracket = duration // 1200
        time_to_billable = 1200 - (duration % 1200)
        percent_billed = (duration % 1200) / 1200 
        if billing_bracket >= 1: # Billable periods after the first period. Billing code: 99458
            dollars_per_minutes = 40.19 / (time_to_billable/60)
        else: # this is the first billable period. Code: 99457
            dollars_per_minutes = 40.19 / (time_to_billable/60)
        return [dollars_per_minutes, time_to_billable/60, time_to_billable, percent_billed]

    elif program == "PCM":
        if duration >= 5400: # this is the maxed out case
            return [0, 0]
        billing_bracket = duration // 1800
        time_to_billable = 1800 - (duration % 1800)
        percent_billed = (duration % 1800) / 1800 
        if billing_bracket >= 1: # Billable periods after the first period. Billing code: 99458
            dollars_per_minutes = 53.47 / (time_to_billable/60)
        else: # this is the first billable period. Code: 99457
            dollars_per_minutes = 78.60 / (time_to_billable/60)
        return [dollars_per_minutes, time_to_billable/60, time_to_billable, percent_billed]

    elif program == "CCM":
        if (duration/60) >= 61:
            billing_bracket = duration // 1800
            time_to_billable = 1800 - (duration % 1800)
            percent_billed = (duration % 1800) / 1800 
            if billing_bracket >= 1:
                dollars_per_minutes = 72.12 / (time_to_billable/60)
            else:
                dollars_per_minutes = 133.93 / (time_to_billable/60)
            return [dollars_per_minutes, time_to_billable/60, time_to_billable, percent_billed]
        else:
            billing_bracket = duration // 1200
            time_to_billable = 1200 - (duration % 1200)
            percent_billed = (duration % 1200) / 1200 
            if billing_bracket >= 1:
                dollars_per_minutes = 47.88 / (time_to_billable/60)
            else:
                dollars_per_minutes = 62.55 / (time_to_billable/60)
            return [dollars_per_minutes, time_to_billable/60, time_to_billable, percent_billed]
    else:
        return [0,0, 0, 0]



patients_by_duration[["revenue per minute", "remaining minutes", "time to billable", "percent billed"]] = patients_by_duration.apply(lambda x: revenue_per_minute(x["Enrolled Care Program"], x["duration in seconds"]), axis=1, result_type="expand")
patients_by_duration.sort_values(by="revenue per minute", ascending=False, inplace=True)
patients_by_duration.head(15)

,Patient Id,Enrolled Care Program,duration in seconds,revenue per minute,remaining minutes,time to billable,percent billed
3419,21065.0,CCM,2220,15.960000,3.0,180.0,0.850000
3579,60786.0,PCM,1500,15.720000,5.0,300.0,0.833333
2560,7411.0,PCM,1500,15.720000,5.0,300.0,0.833333
1955,6397.0,RPM,1020,13.396667,3.0,180.0,0.850000
3427,22324.0,CCM,900,12.510000,5.0,300.0,0.750000
1277,5620.0,CCM,900,12.510000,5.0,300.0,0.750000
2869,8028.0,CCM,900,12.510000,5.0,300.0,0.750000
1287,5628.0,CCM,900,12.510000,5.0,300.0,0.750000
632,4486.0,CCM,900,12.510000,5.0,300.0,0.750000
1925,6367.0,CCM,900,12.510000,5.0,300.0,0.750000


### I want to see how many patients each provider, pod, and User Name has the highest # of people with patients billed > 50% 

In [7]:
merged_df = pd.merge(patients_by_duration, 
                     analytics_assessment[['Patient Id', 'Pharmacy Name', 'POD', 'User Name (First then Last)']].drop_duplicates(), 
                     on="Patient Id", 
                     how="left") # merge in the Pharmacy Name, POD, and User Name

filtered_df = merged_df[merged_df['percent billed'] > 0.5]

In [8]:
user_patient_billed_over_50_perc = filtered_df.groupby(['User Name (First then Last)'])['Patient Id'].nunique().reset_index()

# Rename the column to indicate it's a count of patients
user_patient_billed_over_50_perc.rename(columns={'Patient Id': 'patient_count'}, inplace=True)
user_patient_billed_over_50_perc.sort_values(by=["patient_count"], inplace=True, ascending=False)

user_patient_billed_over_50_perc.head()

,User Name (First then Last),patient_count
14,Karla Miranda,83
15,Mariel Parra,65
3,Annegelie Rivera,49
16,MaryJane DelaCruz,45
7,Danielle Arcete,26


In [9]:
pod_patient_billed_over_50_perc = filtered_df.groupby(['POD'])['Patient Id'].nunique().reset_index()

# Rename the column to indicate it's a count of patients
pod_patient_billed_over_50_perc.rename(columns={'Patient Id': 'patient_count'}, inplace=True)
pod_patient_billed_over_50_perc.sort_values(by=["patient_count"], inplace=True, ascending=False)

pod_patient_billed_over_50_perc.head()


,POD,patient_count
4,Morgen Pod,141
1,Gabby Pod,98
5,Natasha Pod,26
0,Alex Pod,22
6,Wellness POD,6


In [10]:
pharmacy_patient_billed_over_50_perc = filtered_df.groupby(['Pharmacy Name'])['Patient Id'].nunique().reset_index()

# Rename the column to indicate it's a count of patients
pharmacy_patient_billed_over_50_perc.rename(columns={'Patient Id': 'patient_count'}, inplace=True)
pharmacy_patient_billed_over_50_perc.sort_values(by=["patient_count"], inplace=True, ascending=False)

pharmacy_patient_billed_over_50_perc.head()


,Pharmacy Name,patient_count
0,iVira,369


# Employees hours as a share of their pods

In [11]:
df = pd.read_excel("data/Analytics Assessment Report 2024-07-01 2024-07-31.xlsx")
df.rename(columns={"User Name (First then Last)":"Employee"}, inplace=True)
df["duration in seconds"] = df["Duration (exact)"].apply(duration_to_seconds)
df.drop(["EHR#", "FirstName", "LastName", "Birth Date", "Assessment ID", "Tertiary Plan Name"], axis=1, inplace=True)
df.head()

,Patient Id,Primary Plan Name,Secondary Plan Name,Practice,Provider from Practice,Pharmacy Name,Employee,POD,User Type,Duration (exact),Enrolled Care Program,Call Type Selected,Assessment Time Date,duration in seconds
0,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Francis Fuentes,NaN,caremanager,0,CCM,Administrative Tasks,07-01-2024,0
1,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
2,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
3,2612.0,test planHighmark Blue Cross And Blue Shield,TEST Highmark Health Options,Highmark Health Options,Mayda Melendez,iVira,Jay Patel,Alex Pod,pharmacist,0h 00m 49s,PCM,GH Deficiency Care Journey,07-29-2024,49
4,2612.0,test planHighmark Blue Cross And Blue Shield,TEST Highmark Health Options,Highmark Health Options,Mayda Melendez,iVira,Jay Patel,Alex Pod,pharmacist,0h 00m 38s,CCM,Acute Myocardial Infarction Care Journey,07-03-2024,38


In [12]:
#print(df[df["Employee"] == "Gabriella McGinley"].groupby(by=["POD"])["duration in seconds"].sum())
gab_mcginley_df = df[df["Employee"] == "Michelle Mazry"]#[["Patient Id", "Duration (exact)"]]
gab_mcginley_df.head(20)

,Patient Id,Primary Plan Name,Secondary Plan Name,Practice,Provider from Practice,Pharmacy Name,Employee,POD,User Type,Duration (exact),Enrolled Care Program,Call Type Selected,Assessment Time Date,duration in seconds
1,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
2,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
15,2616.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-09-2024,1200
16,2616.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-09-2024,1200
68,2676.0,Amerihealth,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-15-2024,1200
69,2676.0,Amerihealth,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-15-2024,1200
164,2805.0,Amerihealth,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-19-2024,1200
166,2805.0,Amerihealth,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-19-2024,1200
350,3057.0,NaN,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-22-2024,1200
351,3057.0,NaN,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-22-2024,1200


In [13]:
pod_total_seconds_billed = df.groupby(by=["POD"])["duration in seconds"].sum().reset_index()
pod_total_seconds_billed.head()

,POD,duration in seconds
0,Alex Pod,242158
1,Gabby Pod,271500
2,Market Pod,68640
3,Milford Pod,210300
4,Morgen Pod,967560


In [14]:
total_billables_df = {}

def calculate_total_billables_per_pod(employee, pod, duration):
    key = (employee, pod)
    if key in total_billables_df:
        total_billables_df[key] += int(duration)
    else:
        total_billables_df[key] = int(duration)


for i in range(len(df)):
    calculate_total_billables_per_pod(df.iloc[i]["Employee"], df.iloc[i]["POD"], df.iloc[i]["duration in seconds"])

print(total_billables_df)

{('Francis Fuentes', nan): 42600, ('Michelle Mazry', 'Milford Pod'): 177000, ('Michelle Mazry', 'Alex Pod'): 177000, ('Jay Patel', 'Alex Pod'): 5158, ('Euro Montero', 'Gabby Pod'): 90900, ('Natasha Bidadi', 'Natasha Pod'): 392664, ('Danielle Arcete', nan): 218160, ('Mariel Parra', 'Gabby Pod'): 153900, ('Paula Dayandayan', nan): 212100, ('Gabriella McGinley', 'Alex Pod'): 26700, ('Gabriella McGinley', 'Gabby Pod'): 26700, ('Gabriella McGinley', 'Morgen Pod'): 26700, ('AnaCristina Kolster', nan): 147480, ('Brooke Bunting', nan): 58200, ('Zainab Haque', 'Wellness POD'): 119400, ('Diana Muslemani', 'Natasha Pod'): 184500, ('Alycia Lewis', nan): 99900, ('Chelsea Myer', 'Wellness POD'): 97200, ('Alexis Veres', nan): 36300, ('Annegelie Rivera', nan): 209112, ('MaryJane DelaCruz', nan): 358200, ('Victor Guerra', nan): 225900, ('Alex PerryFerrari', 'Milford Pod'): 33300, ('Alex PerryFerrari', 'Alex Pod'): 33300, ('Crystal Lam', nan): 43200, ('Karla Miranda', 'Morgen Pod'): 338110, ('Dina Moura

In [15]:
print(total_billables_df)
result_df = pd.DataFrame([
    {'Employee': emp_dept[0], 'POD': emp_dept[1], 'SecondsWorked': hours}
    for emp_dept, hours in total_billables_df.items()
]).reset_index()
print(result_df)

{('Francis Fuentes', nan): 42600, ('Michelle Mazry', 'Milford Pod'): 177000, ('Michelle Mazry', 'Alex Pod'): 177000, ('Jay Patel', 'Alex Pod'): 5158, ('Euro Montero', 'Gabby Pod'): 90900, ('Natasha Bidadi', 'Natasha Pod'): 392664, ('Danielle Arcete', nan): 218160, ('Mariel Parra', 'Gabby Pod'): 153900, ('Paula Dayandayan', nan): 212100, ('Gabriella McGinley', 'Alex Pod'): 26700, ('Gabriella McGinley', 'Gabby Pod'): 26700, ('Gabriella McGinley', 'Morgen Pod'): 26700, ('AnaCristina Kolster', nan): 147480, ('Brooke Bunting', nan): 58200, ('Zainab Haque', 'Wellness POD'): 119400, ('Diana Muslemani', 'Natasha Pod'): 184500, ('Alycia Lewis', nan): 99900, ('Chelsea Myer', 'Wellness POD'): 97200, ('Alexis Veres', nan): 36300, ('Annegelie Rivera', nan): 209112, ('MaryJane DelaCruz', nan): 358200, ('Victor Guerra', nan): 225900, ('Alex PerryFerrari', 'Milford Pod'): 33300, ('Alex PerryFerrari', 'Alex Pod'): 33300, ('Crystal Lam', nan): 43200, ('Karla Miranda', 'Morgen Pod'): 338110, ('Dina Moura

In [16]:
df.head()

,Patient Id,Primary Plan Name,Secondary Plan Name,Practice,Provider from Practice,Pharmacy Name,Employee,POD,User Type,Duration (exact),Enrolled Care Program,Call Type Selected,Assessment Time Date,duration in seconds
0,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Francis Fuentes,NaN,caremanager,0,CCM,Administrative Tasks,07-01-2024,0
1,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Milford Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
2,2583.0,Highmark Blue Cross And Blue Shield,NaN,Ivira Pharmacy,Mayda Melendez,iVira,Michelle Mazry,Alex Pod,caremanager,0h 20m 00s,CCM,Administrative Tasks,07-08-2024,1200
3,2612.0,test planHighmark Blue Cross And Blue Shield,TEST Highmark Health Options,Highmark Health Options,Mayda Melendez,iVira,Jay Patel,Alex Pod,pharmacist,0h 00m 49s,PCM,GH Deficiency Care Journey,07-29-2024,49
4,2612.0,test planHighmark Blue Cross And Blue Shield,TEST Highmark Health Options,Highmark Health Options,Mayda Melendez,iVira,Jay Patel,Alex Pod,pharmacist,0h 00m 38s,CCM,Acute Myocardial Infarction Care Journey,07-03-2024,38


In [19]:
billed_hours_by_care_type = df.groupby(by=["Call Type Selected"])["duration in seconds"].sum().reset_index()
billed_hours_by_care_type["duration in minutes"] = round(billed_hours_by_care_type["duration in seconds"] / 60, 2)
billed_hours_by_care_type.head()

,Call Type Selected,duration in seconds,duration in minutes
0,AMR Call,3300,55.000000
1,"AMR Call, Atrial Fibrillation and Atrial Flutt...",29,0.483333
2,Acute Myocardial Infarction Care Journey,3855,64.250000
3,Administrative Tasks,1146932,19115.533333
4,"Administrative Tasks, Depression Care Journey",12600,210.000000
